# Initial Value Problems for Ordinary Differential Equations, Part 5: Error Control and Variable Step Sizes — Preliminary Version

**Version of April 5, 2021**

**References:**

- Section 6.5 *Variable Step Size Methods* of [Sauer](../references.html#Sauer)

- Section 5.5 *Error Control and the Runge-Kutta-Fehlberg Method* of [Burden&Faires](../references.html#Burden-Faires)

- Section  7.3 of [Chenney&Kincaid](../references.html#Chenney-Kincaid)

## The Basic ODE Initial Value Problem

We consider again the initial value problem

$$
\frac{d u}{d t} = f(t, u) \quad a \leq t \leq b, \quad u(a) = u_0
$$

We now allow the possibility that $u$ and $f$ are vector-valued as in the section on
[Systems of ODEs and Higher Order ODEs](ODE-IVP-4-system-higher-order-equations.ipynb),
but omitting the tilde notation $\tilde u$, $\tilde f$.

## Error Control by Varying the Time Step Size $h_i$

Recall the variable step-size version of Euler's method:

Input: $f$, $a$, $b$, $n$ <br>

$t_0 = a$ <br>
$U_0 = u_0$ <br>
$h = (b-a)/n$ <br>

for i in $[0, n)$:
<br>

$\qquad$ Choose step size $h_i$ somehow!
<br>

$\qquad$ $t_{i+1} = t_i + h_i$
<br>

$\qquad$ $U_{i+1} = U_i + h_i f(t_i, U_i)$
<br>

end for

We now consider how to choose each step size, by estimating the error in each step, and aiming to have error per unit time below some limit like $\epsilon/(b-a)$, so that the global error is no more than about $\epsilon$.

As usual, the theoretical error bounds like $O(h_i^2)$ for a single step of Euler's method are not enough for quantitative tasks like choosing $h_i$, but they do motivate more practical estimates.

## A crude error estimator for Euler's Method: Richardson Extrapolation

Starting at a point (t, u(t)), we can estimate the error in Euler's method approximato at a slightly later time $t_i + h$ by using two approximations of $U(t + h)$:
- The value given by a step of Euler's method with step size $h$: call this $U^{h}$
- The value given by taking two steps of Euler's method each with step size $h/2$: call this $U_2^{h/2}$,
because it involves 2 steps of size $h/2$.

The first order accuracy of Euler's method gives $e_h := u(t+h) - U^{h} \approx 2(u(t+h) - U_2^{h/2})$,
so that

$$e_h \approx \frac{U_2^{h/2} - U^{h}}{2}$$

### Step size choice

What do we do with this error information?

The first obvious ideas are:
- Accept this step if $e_h$ is small enough, taking $h_i = h$, $t_{i+1} = t_i + h_i$, and $U_{i+1} = U^h$, but
- reject it and try again with a smaller $h$ value otherwise; maybe halving $h$; but there are more sophisticated options too.

### Exercise 1

Write a formula for $U_h$ and $e_h$ if one starts from the point $(t_i, U_i)$, so that $(t_i + h, U^h)$ is the proposed value for the next point $(t_{i+1}, U_{i+1})$ in the approximate solution — but only if $e_h$ is small enough!

### Error tolerance 

One simple criterion for accuracy is that the estimated error in this step be no more than some overall upper limit on the error in ech time step, $\epsilon$.
That is, accept the step size $h$ if

$$|e_h| \leq \epsilon.$$

### A crude approach to reducing the step size when needed

If this error tolerance is not met, we must choose a new step size $h'$, and we can predict roughly its error behavior using the known order natue of the error in Euler's method: scaling dowen to $h' = s h$, the error in a step scales with $h^2$, so the error per unit time scales with $h$ (in general it scales with $h^p$ for a method of order $p$), and so to reduce the error by the needed factor $\displaystyle \frac{e_h/h}{\epsilon}$ one needs approximately

$$
s = \frac{\epsilon}{|e_h|/h} = \frac{h \epsilon}{|e_h|}
$$

and so using $e_h \approx |U^{h/2} - U^{h}|$ suggests using

$$
\begin{split}
s &= \frac{h \epsilon}{|U^{h/2} - U^{h}|}
\\
h' &= sh = \frac{2 h^2 \epsilon}{|U^{h/2} - U^{h}|}.
\end{split}
$$

However this new step size might have error that is still slightly too large, leading to a second failure.
Another is that one might get into an infinite loop of step size reduction.

So refinements of this choice must be considered.

### Increasing the step size when desirable

If we simply follow the above aproach, the step size, once reduced, will never be increased.
This could lead to great inefficiency, through using an unecessarily small step size just because at an earlier part of the time domain, accuracy required very small steps.

Thus, after a successful time step, one might consider increasing $h$ for the next step.
This could be done using exactly the above formula, but again there are risks, so again refinement of this choice must be considered.

One problem is that if the step size gets too large, the error estimate can become unreliable; another is that one might need some minimum "temporal resolution", for nice graphs and such.

Both suggest imposing an upper limit on the step size $h$.

## Another strategy for getting error estimates: two (related) methods

The recurring strategy of estimating errors by the difference of two different approximations — one expected to be far better than the other — can be used in a nice way here.
I iwl ilusrat frst with the smplset version, using Euler's Matha dnt explicit Trapezoid Method.

Revsl th the increment in Euler's Method from time $t$ to time $t+h$ is

$$K_1 = h f(t, u)$$

whereas for the explict trapezoid method it is $(K_1 + K_2)/2$, as given by

$$\begin{split}
K_1 &= h f(t_i, U_i)
\\
K_2 &= h f(t_{i+1}, U_i + K_1)
\end{split}$$

Thus we can use the difference, $|K_1 - (K_1 + K_2)/2| = |(K_1 - K_2)/2|$ as an error estimate.
In fact to be cautious, one often drops the factor of $1/2$, so using $e_h \approx |K_1 - K_2|$.

One has to be careful: this estimates the error in Euler's Method, so one has to use it that way: using the less accurate value $K_1$ as the update.

One basic algorithm for the time step starting with $t_i, U_i$ is

$K_1 \leftarrow h f(t_i, U_i)$

$K_2 \leftarrow f(t_{i+1}, U_i + K_1)$

$e_{h} \leftarrow |K_1 - K_2|$

$s \leftarrow (h \epsilon)/e_h$

if $s>1$: $\qquad$ (I.e.  $e_h/h < \epsilon$ and so good enough)

$\quad U_{i+1} = U_i + K_1$

$\quad t_{i+1} = t_i + h$

$\quad$ Increase $h$ for the *next* time step:

$\quad h \leftarrow s h$

else: $\qquad$ ($s \leq 1$, so not good enough: reduce $h$ and try again)

$\quad h \leftarrow s h$

$\quad$ Start again from $K_1 = \dots$

end if

However, in practice one needs:

- An upper limit $h_{max}$ on the step size $h$, partly becuase eror estimates are unreliable if $h$ is too large,
and subsequent use of the results (like graphs) might need sufficiently "fine" data.

- A lower limit $h_{max}$ on $h$, to avoid infinite loops and such.

Incorporating these refinements:

$K_1 \leftarrow h f(t_i, U_i)$

$K_2 \leftarrow f(t_{i+1}, U_i + K_1)$

$e_{h} \leftarrow |K_1 - K_2|$

$s = \epsilon/(e_h/h)$

if $s>1$: $\qquad$ (I.e.  $e_h/h < \epsilon$ and so good enough)

$\quad U_{i+1} = U_i + K_1$

$\quad t_{i+1} = t_i + h$

$\quad$ Increase $h$ for the *next* time step:

$\quad h \leftarrow \min(s h, h_{max})$

else: $\qquad$ ($s \leq 1$, so not good enough; reduce $h$ and try again)

$\quad h \leftarrow \max(s h, h_{min})$

$\quad$ Start again from $K_1 = \dots$

end if

### Exercise 2

Implement the above, and test on the two familiar examples

$$
\begin{split}
du/dt &= Ku
\\
&\text{and}
\\
du/dt &= K(\cos(t) - u) - \sin(t)
\end{split}
$$

($K=1$ is enough.)

## Second Order Accurate Methods with Error Control

In practice, one usually needs at least second order accuracy, and one approach to that is using computing a "candidates" for teh next time step with a second order accurate Runge-Kutta method and also a third order accurate one, the latter used only to get an error estimate for the former.

Perhaps the simplest of these is based on adding error estimation to the Explicit Trapezoid Rule;
omitting the step sze adjustment for now, the main ingredients are:

$K_1 = h f(t_i, U_i)$

$K_2 = f(t_i + h, U_i + K_1)$

(So far, as for the explicit trapezoid method)

$K_3 = f(t_i + h/2, U_i + (K_1+K_2)/2)$

$\delta_2 = (K_1 + K_2)/2$
(The order 2 increment as for the explicit trapezoid method)

$\delta_3 = (K_1 + 4 K_3 + K_2)/6$
(An order 3 increment — note the resemblance to Simpson's Method)

$e_h = |\delta_2 - \delta_3 |, \, = (K_1 -2 K_3 + K_2)/3$

Again, if this step is accepted, one uses the explicit trapezoid rule step: $U_{i+1} = U_i + \delta_2$.

### Step size adjustment

The scale factor $s$ for step size adjustment is a bit different; for a method order $p$ (so $p=2$ now)
changing step size by a factor $s$ will change the error $e_h$ in a single time step by a factor of about $s^{p+1}$.

Thus, we want a new step with this rescaled error of about $s^{p+1} e_h$ roughly matching the tolerance $\epsilon$.
Equating would give $s^{p+1} e_h/h = \epsilon$ so that $ = (\epsilon/(e_h/h))^{1/(p+1)}$

$$s = (\epsilon/(e_h/h))^{1/p}$$

## Fourth Order Accurate Methods with Error Control: Runge-Kutta-Felberg and Some Newer Refinements

The details involve some messy coefficients; see the references above for those.

The basic idea is that ...